In [ ]:
import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
from sklearn.metrics import accuracy_score

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt_tab')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("translated_data_train_32k.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32535 entries, 0 to 32534
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   original      32535 non-null  object
 1   translated    32534 non-null  object
 2   cleaned_text  32530 non-null  object
 3   label         32535 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1016.8+ KB


In [ ]:
df = df.dropna(subset=['cleaned_text'])


In [ ]:
df.isnull().sum()

,0
original,0
translated,0
cleaned_text,0
label,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32530 entries, 0 to 32534
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   original      32530 non-null  object
 1   translated    32530 non-null  object
 2   cleaned_text  32530 non-null  object
 3   label         32530 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [ ]:
df.head()

,original,translated,cleaned_text,label
0,"My father - all of my life - has shifted between a wonderful person and a monster. At his best, he is supportive, calm, positive, funny, caring. At his worst, he is vicious, selfish, hateful, paranoid, mean, abusive, and hurtful. He has become an alcoholic slowly in the past decade. I believe he has BPD - but has blatantly and combatively refused to get help everytime it has been brought up in the past.\n\nFast forward - I carried a lot of baggage on my shoulder which led to quite a bit of self medication. I've been relatively successful in my career choice, although it is an arduous journey in my chosen profession. A year ago, I decided to move in with my grandparents to prepare for doctoral school, find a job to get some finances in order, and get the monkey off of my back. \n\nThat summer, he was offended I wasn't going to him for help (which of course, he can not provide, and would do far more to destruct me than help me). He wanted me to work with him (he does independent landscaping) and when I refused to work with him, he lashed out. In a drunken stupor, he called me and left over 15 voice mails that were viscous and disgusting. He is excellent at finding people's weaknesses and pouncing on them. Again, something he has done all of my life. After a few days, I called him and told him if he ever talked to me that way again, it would be the last time he talked to me.\n\nWell, fast forward a year and half. I've moved away to another state to start doctoral school and am free from the burdens I have had in the past. I am doing very well at school, although I still sometimes deal with residual of the past. My dad started doing it again. He called on a Sunday night. On Monday at 8am, I teach a class at my university. So I go to bed relatively early, but at 11:30 he started texting and calling me. I told him I had to get up early, knowing he was drunk (after listening to a voice mail), that he needed to stop. This escalated everything. I don't know what got in to me, but I decided to escalate him as far as I could go. I threw, for once, everything he gave me right back at him. I told him, that night, there would be consequences - I reminded him of my ultimatum, but he would not stop. I blocked his number, and his voice mails still went to my inbox, eventually filling it up.\n\nSo HERE's MY question...finally...I have told him (after a week of getting terrible text messages and phone calls) that I will not contact him until he has seen a doctor, started therapy, and sought out medication. \n\nCan someone, or many of you, please help me here? This is something he NEEDS - or I fear he will either commit suicide or drink himself to death. Both have been fears of the past. How do I help him, but at the same time retain and reinforce my boundaries that he has torn down my entire life in efforts to control me. \n\nPlease. Halp.","Ayah saya - sepanjang hidup saya - telah bergeser antara orang yang luar biasa dan monster. Yang terbaik, dia mendukung, tenang, positif, lucu, peduli. Paling buruknya, dia kejam, egois, penuh kebencian, paranoid, kejam, kasar, dan menyakitkan. Dia telah menjadi pecandu alkohol perlahan dalam dekade terakhir. Saya percaya dia memiliki BPD - tetapi telah secara terang -terangan dan secara agung menolak untuk mendapatkan bantuan setiap kali telah dibesarkan di masa lalu.\n\nMaju cepat - Saya membawa banyak bagasi di bahu saya yang menyebabkan cukup banyak obat diri. Saya relatif sukses dalam pilihan karier saya, meskipun ini adalah perjalanan yang sulit dalam profesi pilihan saya. Setahun yang lalu, saya memutuskan untuk pindah dengan kakek -nenek saya untuk mempersiapkan sekolah doktoral, mencari pekerjaan untuk mendapatkan beberapa keuangan, dan mendapatkan monyet dari punggung saya. \n\nMusim panas itu, dia tersinggung saya tidak akan meminta bantuan kepadanya (yang tentu saja, dia tidak bisa menyediakan, dan akan melakukan lebih banyak untuk menghancurkan saya dar

In [ ]:
# --- Fungsi-fungsi ---

# Inisialisasi stemmer di luar fungsi supaya tidak dibuat ulang berkali-kali
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def casefoldingText(text):  # Ubah huruf jadi kecil semua
    return text.lower()

def tokenizingText(text):  # Tokenisasi kata
    return word_tokenize(text)

def filteringText(tokens):  # Hapus stopwords
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords.update(stopwords.words('english'))
    listStopwords.update(['iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', "di", "ga", "ya", "gaa", "loh", "kah", "woi", "woii", "woy",'tetapi', 'orang', 'akan', 'benar', 'karena', 'merasa', 'sangat', 'bisa',
 'apa', 'seperti', 'hanya', 'tahu', 'memiliki', 'hal', 'bahwa', 'anda',
 'ada', 'ingin', 'lebih', 'dalam', 'diri', 'baik', 'telah', 'hari',
 'menjadi', 'ketika', 'tentang', 'membuat', 'banyak', 'jika'
])
    return [word for word in tokens if word not in listStopwords]

def stemmingText(tokens):  # Stemming per kata, input dan output list token
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return stemmed_words

def toSentence(list_words):  # Gabungkan token jadi kalimat
    return ' '.join(word for word in list_words)


In [ ]:
from tqdm import tqdm
tqdm.pandas()  # untuk mengaktifkan progress_apply

df['casefolded'] = df['cleaned_text'].progress_apply(casefoldingText)
df['tokenized'] = df['casefolded'].progress_apply(tokenizingText)
df['filtered'] = df['tokenized'].progress_apply(filteringText)
df['stemmed'] = df['filtered'].progress_apply(stemmingText)
df['final_sentence'] = df['stemmed'].progress_apply(toSentence)

100%|██████████| 32530/32530 [00:00<00:00, 222311.00it/s]


In [ ]:
df.head()

,original,translated,cleaned_text,label,casefolded,tokenized,filtered,stemmed,final_sentence
0,"My father - all of my life - has shifted between a wonderful person and a monster. At his best, he is supportive, calm, positive, funny, caring. At his worst, he is vicious, selfish, hateful, paranoid, mean, abusive, and hurtful. He has become an alcoholic slowly in the past decade. I believe he has BPD - but has blatantly and combatively refused to get help everytime it has been brought up in the past.\n\nFast forward - I carried a lot of baggage on my shoulder which led to quite a bit of self medication. I've been relatively successful in my career choice, although it is an arduous journey in my chosen profession. A year ago, I decided to move in with my grandparents to prepare for doctoral school, find a job to get some finances in order, and get the monkey off of my back. \n\nThat summer, he was offended I wasn't going to him for help (which of course, he can not provide, and would do far more to destruct me than help me). He wanted me to work with him (he does independent landscaping) and when I refused to work with him, he lashed out. In a drunken stupor, he called me and left over 15 voice mails that were viscous and disgusting. He is excellent at finding people's weaknesses and pouncing on them. Again, something he has done all of my life. After a few days, I called him and told him if he ever talked to me that way again, it would be the last time he talked to me.\n\nWell, fast forward a year and half. I've moved away to another state to start doctoral school and am free from the burdens I have had in the past. I am doing very well at school, although I still sometimes deal with residual of the past. My dad started doing it again. He called on a Sunday night. On Monday at 8am, I teach a class at my university. So I go to bed relatively early, but at 11:30 he started texting and calling me. I told him I had to get up early, knowing he was drunk (after listening to a voice mail), that he needed to stop. This escalated everything. I don't know what got in to me, but I decided to escalate him as far as I could go. I threw, for once, everything he gave me right back at him. I told him, that night, there would be consequences - I reminded him of my ultimatum, but he would not stop. I blocked his number, and his voice mails still went to my inbox, eventually filling it up.\n\nSo HERE's MY question...finally...I have told him (after a week of getting terrible text messages and phone calls) that I will not contact him until he has seen a doctor, started therapy, and sought out medication. \n\nCan someone, or many of you, please help me here? This is something he NEEDS - or I fear he will either commit suicide or drink himself to death. Both have been fears of the past. How do I help him, but at the same time retain and reinforce my boundaries that he has torn down my entire life in efforts to control me. \n\nPlease. Halp.","Ayah saya - sepanjang hidup saya - telah bergeser antara orang yang luar biasa dan monster. Yang terbaik, dia mendukung, tenang, positif, lucu, peduli. Paling buruknya, dia kejam, egois, penuh kebencian, paranoid, kejam, kasar, dan menyakitkan. Dia telah menjadi pecandu alkohol perlahan dalam dekade terakhir. Saya percaya dia memiliki BPD - tetapi telah secara terang -terangan dan secara agung menolak untuk mendapatkan bantuan setiap kali telah dibesarkan di masa lalu.\n\nMaju cepat - Saya membawa banyak bagasi di bahu saya yang menyebabkan cukup banyak obat diri. Saya relatif sukses dalam pilihan karier saya, meskipun ini adalah perjalanan yang sulit dalam profesi pilihan saya. Setahun yang lalu, saya memutuskan untuk pindah dengan kakek -nenek saya untuk mempersiapkan sekolah doktoral, mencari pekerjaan untuk mendapatkan beberapa keuangan, dan mendapatkan monyet dari punggung saya. \n\nMusim panas itu, dia tersinggung saya tidak akan meminta bantuan kepadanya (yang tentu saja, dia tidak bisa menyediakan, dan aka

In [ ]:
df.to_csv('preprocessed_dataset.csv', index=False)

In [ ]:
df['label'].value_counts()

,count
label,
0,11827
3,9249
2,6666
5,2168
1,1991
4,629


In [ ]:
df2=pd.read_csv('translated_data_test.csv')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23190 entries, 0 to 23189
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   original      23190 non-null  object
 1   translated    23188 non-null  object
 2   cleaned_text  23184 non-null  object
 3   label         23190 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 724.8+ KB


In [ ]:
df2.isnull().sum()

,0
original,0
translated,2
cleaned_text,6
label,0


In [ ]:
df2 = df2.dropna(subset=['cleaned_text'])


In [ ]:
df2.isnull().sum()

,0
original,0
translated,0
cleaned_text,0
label,0


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23184 entries, 0 to 23189
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   original      23184 non-null  object
 1   translated    23184 non-null  object
 2   cleaned_text  23184 non-null  object
 3   label         23184 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 905.6+ KB


In [ ]:
df2.head()

,original,translated,cleaned_text,label
0,And I think that's the worst part. The BPD alr...,Dan saya pikir itu bagian terburuk. BPD sudah ...,Dan saya pikir itu bagian terburuk BPD sudah m...,0
1,How do you deal with it when you’re already de...,Bagaimana Anda menghadapinya saat Anda sudah t...,Bagaimana Anda menghadapinya saat Anda sudah t...,2
2,**im not going to do anything to myself nor am...,** Saya tidak akan melakukan apa pun untuk dir...,Saya tidak akan melakukan apa pun untuk diri s...,0
3,What causes or tiggers panic attacks when you ...,Apa yang menyebabkan atau menyerang Panik Tigg...,Apa yang menyebabkan atau menyerang Panik Tigg...,3
4,I turned 21 about a month ago and I took on ab...,Saya berusia 21 tahun sekitar sebulan yang lal...,Saya berusia 21 tahun sekitar sebulan yang lal...,2


In [ ]:
from tqdm import tqdm
tqdm.pandas()  # ini untuk mengaktifkan progress_apply

df2['casefolded'] = df2['cleaned_text'].progress_apply(casefoldingText)
df2['tokenized'] = df2['casefolded'].progress_apply(tokenizingText)
df2['filtered'] = df2['tokenized'].progress_apply(filteringText)
df2['stemmed'] = df2['filtered'].progress_apply(stemmingText)
df2['final_sentence'] = df2['stemmed'].progress_apply(toSentence)

100%|██████████| 23184/23184 [00:00<00:00, 193827.51it/s]


In [ ]:
df2.head()

,original,translated,cleaned_text,label,casefolded,tokenized,filtered,stemmed,final_sentence
0,And I think that's the worst part. The BPD alr...,Dan saya pikir itu bagian terburuk. BPD sudah ...,Dan saya pikir itu bagian terburuk BPD sudah m...,0,dan saya pikir itu bagian terburuk bpd sudah m...,"[dan, saya, pikir, itu, bagian, terburuk, bpd,...","[pikir, terburuk, bpd, berjuang, pengabaian, m...","[pikir, buruk, bpd, juang, abai, alami, episod...",pikir buruk bpd juang abai alami episode erti ...
1,How do you deal with it when you’re already de...,Bagaimana Anda menghadapinya saat Anda sudah t...,Bagaimana Anda menghadapinya saat Anda sudah t...,2,bagaimana anda menghadapinya saat anda sudah t...,"[bagaimana, anda, menghadapinya, saat, anda, s...","[menghadapinya, tertekan]","[hadap, tekan]",hadap tekan
2,**im not going to do anything to myself nor am...,** Saya tidak akan melakukan apa pun untuk dir...,Saya tidak akan melakukan apa pun untuk diri s...,0,saya tidak akan melakukan apa pun untuk diri s...,"[saya, tidak, akan, melakukan, apa, pun, untuk...","[berpikir, tinggal, tunangan, satunya, setahun...","[pikir, tinggal, tunang, satu, tahun, omong, k...",pikir tinggal tunang satu tahun omong kosong p...
3,What causes or tiggers panic attacks when you ...,Apa yang menyebabkan atau menyerang Panik Tigg...,Apa yang menyebabkan atau menyerang Panik Tigg...,3,apa yang menyebabkan atau menyerang panik tigg...,"[apa, yang, menyebabkan, atau, menyerang, pani...","[menyebabkan, menyerang, panik, tiggers, bangu...","[sebab, serang, panik, tiggers, bangun, alami,...",sebab serang panik tiggers bangun alami lelah ...
4,I turned 21 about a month ago and I took on ab...,Saya berusia 21 tahun sekitar sebulan yang lal...,Saya berusia 21 tahun sekitar sebulan yang lal...,2,saya berusia 21 tahun sekitar sebulan yang lal...,"[saya, berusia, 21, tahun, sekitar, sebulan, y...","[berusia, 21, sebulan, mengambil, 5, kelas, te...","[usia, 21, bulan, ambil, 5, kelas, organisir, ...",usia 21 bulan ambil 5 kelas organisir bahagia ...


In [ ]:
df2.to_csv('preprocessed_data_test.csv', index=False)